In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,72,2024-03-26,3,117.690141,100.042254,0.566803,11.781690,19.557746,0.215085,116.716901,...,10.922535,21.754930,0.217690,120.394366,0,3,1,2,14,2024
1,71,2024-03-26,-2,118.342857,99.022857,0.553957,11.828571,27.337143,0.179243,118.521429,...,10.933803,21.794366,0.213704,114.549296,0,3,1,2,10,2024
2,71,2024-03-26,-1,120.342857,99.071429,0.574429,10.474286,20.865714,0.202243,120.764286,...,11.423944,25.035211,0.205549,118.802817,0,3,1,2,21,2024
3,71,2024-03-26,3,118.642857,99.650000,0.563829,10.641429,22.252857,0.197714,119.048571,...,11.225714,23.992857,0.173986,118.005714,0,3,1,5,7,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0
1,2024-03-26,GSW,MIA,1.819672,2.020000,0
2,2024-03-26,OKC,NOP,1.800000,2.050000,0
3,2024-03-26,DAL,SAC,1.980392,1.847458,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0
1,2024-03-26,GSW,MIA,1.819672,2.020000,0
2,2024-03-26,OKC,NOP,1.800000,2.050000,0
3,2024-03-26,DAL,SAC,1.980392,1.847458,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0
1,2024-03-26,GSW,MIA,1.819672,2.020000,0
2,2024-03-26,OKC,NOP,1.800000,2.050000,0
3,2024-03-26,DAL,SAC,1.980392,1.847458,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0
1,2024-03-26,GSW,MIA,1.819672,2.020000,0
2,2024-03-26,OKC,NOP,1.800000,2.050000,0
3,2024-03-26,DAL,SAC,1.980392,1.847458,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0.0
1,2024-03-26,GSW,MIA,1.819672,2.020000,1.0
2,2024-03-26,OKC,NOP,1.800000,2.050000,1.0
3,2024-03-26,DAL,SAC,1.980392,1.847458,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-26,LAL,MIL,4.200000,1.243902,0.0
1,2024-03-26,GSW,MIA,1.819672,2.020000,1.0
2,2024-03-26,OKC,NOP,1.800000,2.050000,1.0
3,2024-03-26,DAL,SAC,1.980392,1.847458,1.0
